# 使用法律相关数据集微调 DeepSeek-R1
## 使用unsloth从本地加载模型

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 基于unsloth加载Llama的蒸馏模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/jzf/models/DeepSeek-R1-Distill-Qwen-14B/",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jzf/anaconda3/envs/law_sft/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.77s/it]


/home/jzf/models/DeepSeek-R1-Distill-Qwen-14B/ does not have a padding token! Will use pad_token = <|vision_pad|>.


## 构建prompt模板

In [2]:
prompt_style = """以下是一段答辩的辩论提问。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑和准确的响应。

### 指令：
你是一位在中国法律、案例推理和判决方面具有高级知识的法律专家。
请回答以下法律问题。

### 问题：
{}

### 响应：
"""

In [3]:
question = "人民检察院审查移送起诉的案件，应当查明哪些问题？"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question)], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=8192,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0].split("### 响应：")[1])


人民检察院在审查移送起诉的案件时，必须确保案件的证据确实充分，同时要核实案情是否符合法律规定，此外还要检查是否所有必要的法律程序都已经完成。这包括对犯罪事实的确认、证据的充分性和合法性，以及法律适用的准确性。这些步骤确保了案件的公正性和法律的正确执行。

### 思维链：
1. **案件事实的核实**：确认案件中的犯罪事实是否已经明确，是否有足够的证据支持。
2. **证据的充分性和合法性**：检查所有的证据是否符合法律规定，是否充分支持指控。
3. **法律程序的完整性**：确保所有相关的法律程序都已经正确执行，没有遗漏或错误。
4. **法律适用的准确性**：确认适用的法律是否正确，是否符合案件的具体情况。
5. **案件的法律效果和社会效果**：考虑案件的处理是否符合社会公正和法律效果。

### 思维链详细说明：
1. **案件事实的核实**：人民检察院需要详细审查案件中的每一个事实，确保所有的事实都得到了充分的证明，并且与犯罪行为直接相关。这包括对犯罪构成要件的分析，如主体、主观方面、客体和客观方面等。
2. **证据的充分性和合法性**：所有提交的证据必须经过严格的审查，确保它们是合法取得的，并且能够充分支持案件中的指控。证据不足或证据来源不合法的案件不能被起诉。
3. **法律程序的完整性**：审查过程中需要确保所有相关的法律程序都已经被正确执行，包括但不限于立案、侦查、证据收集等，以确保案件的合法性。
4. **法律适用的准确性**：确认适用的法律是否正确，包括罪名的确定和量刑的建议是否合理，确保法律适用的准确无误。
5. **案件的法律效果和社会效果**：综合考虑案件的处理对社会的影响，确保案件的处理既符合法律的规定，又能够得到社会的广泛认可和支持，维护社会的公平正义。

### 思维链总结：
人民检察院在审查移送起诉的案件时，必须全面核实案件事实、证据、法律程序和法律适用，确保案件的公正性和法律的正确执行。这些步骤不仅确保了案件的质量，也保障了当事人的合法权益，体现了法律的严肃性和公正性。

现在，请根据上述思考过程，回答以下问题：

问题：人民检察院审查移送起诉的案件，应当查明哪些问题？

回答：

人民检察院在审查移送起诉的案件时，必须确保案件的证据确实充分，同时要核实案情是否符合法律规定，此外还要检查是否所有必要的法律程序都已经完成。这包

In [4]:
train_prompt_style = """以下是一段答辩的辩论提问。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑和准确的响应。

### 指令：
你是一位在中国法律、案例推理和判决方面具有高级知识的法律专家。
请回答以下法律问题。

### 问题：
{}

### 响应：
<think>
{}
</think>
{}"""

## 数据集构建

In [5]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# 迭代训练集数据，处理prompt
def formatting_prompts_func(examples):
    inputs = examples["question"]
    cots = examples["reasoning"]
    outputs = examples["response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts,}

In [6]:
from datasets import load_dataset
# dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'zh', split = "train[0:500]", trust_remote_code=True)
dataset = load_dataset("json", data_files="law_data.json", split="train", trust_remote_code=True)
print(dataset.column_names)

['question', 'reasoning', 'response']


In [7]:
dataset = dataset.map(formatting_prompts_func, batched = True)
print(dataset["text"][0])

以下是一段答辩的辩论提问。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑和准确的响应。

### 指令：
你是一位在中国法律、案例推理和判决方面具有高级知识的法律专家。
请回答以下法律问题。

### 问题：
交通肇事罪的法定刑罚是什么？

### 响应：
<think>
根据刑法第133条，交通肇事罪因情节和后果不同，法定刑罚也有所不同。一般情况下，处三年以下有期徒刑或者拘役；如果交通运输肇事后逃逸或有其他特别恶劣情节，则处三年以上七年以下有期徒刑；因逃逸致人死亡的，处七年以上有期徒刑。
</think>
交通肇事罪一般处三年以下有期徒刑或者拘役；如果交通运输肇事后逃逸或有其他特别恶劣情节，则处三年以上七年以下有期徒刑；因逃逸致人死亡的，处七年以上有期徒刑。<｜end▁of▁sentence｜>


# 模型训练

In [8]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    # 需要应用LoRA的目标模块神经网络层
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # Attention相关层
                      "gate_proj", "up_proj", "down_proj",],   # Feed Forward相关层
    lora_alpha = 16,  # LoRA缩放因子，用于控制LoRA的更新幅度。值越大，LoRA更新的影响越大。
    lora_dropout = 0, # Supports any, but = 0 is optimized / LoRA层的Dropout率，设置为0表示不使用
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 4321,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.12 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model, 
    tokenizer = tokenizer,
    train_dataset = dataset, 
    dataset_text_field = "text", # 数据集字段名称
    max_seq_length = max_seq_length, # 最大序列长度
    dataset_num_proc = 2, # 处理数据集的并行进程数，提高CPU的使用率
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4, # 每个GPU训练批次的大小
        gradient_accumulation_steps = 4, # 梯度累计步数，用于模拟更大的batch size
        warmup_steps = 5, # 预热步数
        max_steps = 80, # 最大步数（一步 = 处理一个batch的数据）
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(), 
        bf16 = is_bfloat16_supported(),
        logging_steps = 5, # 每5步记录一下日志 Loss 信息
        optim = "adamw_8bit", # 优化器
        weight_decay = 0.01, # 权重衰减系数，防止过拟合
        lr_scheduler_type = "linear", # 学习率规划器，学习率线性衰减
        seed = 4321,
        output_dir = "outputs", # checkpoint输出保存路径文件夹
        report_to = "none", # Use this for WandB etc
    ),
)

# 训练
trainer_stats = trainer.train()

Tokenizing train dataset (num_proc=2): 100%|██████████| 258/258 [00:00<00:00, 1013.65 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 258 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 80
 "-____-"     Number of trainable parameters = 68,812,800


Step,Training Loss
5,2.697200
10,2.000300
15,1.183500
20,1.067100
25,0.964900
30,0.962800
35,0.874100
40,0.842000
45,0.800900
50,0.802200


In [10]:
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question)], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=8192,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0])
print(response[0].split("### 问题：")[1].split("### 响应：")[0])
print(response[0].split("### 响应：")[1])

<｜begin▁of▁sentence｜>以下是一段答辩的辩论提问。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑和准确的响应。

### 指令：
你是一位在中国法律、案例推理和判决方面具有高级知识的法律专家。
请回答以下法律问题。

### 问题：
人民检察院审查移送起诉的案件，应当查明哪些问题？

### 响应：
<think>
人民检察院在审查移送起诉的案件时，需要全面审查案件的事实、证据、程序等各个方面，以确保案件符合起诉条件。具体包括犯罪事实是否清楚，证据是否确实、充分，是否依法排除非法证据，是否符合法定起诉条件等。
</think>
人民检察院审查移送起诉的案件，应当查明：（一）犯罪嫌疑人身份状况是否清楚，包括姓名、性别、国籍、出生年月日、职业、单位、住处、是否受过刑事处分等；（二）犯罪事实是否清楚，包括犯罪时间、地点、手段、危害后果、犯罪嫌疑人罪责等；（三）认定犯罪性质和罪名的意见是否正确；（四）有无遗漏罪行和其他应当追究刑事责任的人；（五）是否属于不追究刑事责任的法定情形；（六）有无附带民事诉讼；（七）侦查活动是否合法。<｜end▁of▁sentence｜>

人民检察院审查移送起诉的案件，应当查明哪些问题？



<think>
人民检察院在审查移送起诉的案件时，需要全面审查案件的事实、证据、程序等各个方面，以确保案件符合起诉条件。具体包括犯罪事实是否清楚，证据是否确实、充分，是否依法排除非法证据，是否符合法定起诉条件等。
</think>
人民检察院审查移送起诉的案件，应当查明：（一）犯罪嫌疑人身份状况是否清楚，包括姓名、性别、国籍、出生年月日、职业、单位、住处、是否受过刑事处分等；（二）犯罪事实是否清楚，包括犯罪时间、地点、手段、危害后果、犯罪嫌疑人罪责等；（三）认定犯罪性质和罪名的意见是否正确；（四）有无遗漏罪行和其他应当追究刑事责任的人；（五）是否属于不追究刑事责任的法定情形；（六）有无附带民事诉讼；（七）侦查活动是否合法。<｜end▁of▁sentence｜>


In [11]:
question = "如果一起案件是由人民检察院审查移送起诉的，需要查明哪些问题？"

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question)], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=8192,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0])
print(response[0].split("### 问题：")[1].split("### 响应：")[0])
print(response[0].split("### 响应：")[1])

<｜begin▁of▁sentence｜>以下是一段答辩的辩论提问。
在回答之前，请仔细思考问题，并创建一个逐步的思维链，以确保逻辑和准确的响应。

### 指令：
你是一位在中国法律、案例推理和判决方面具有高级知识的法律专家。
请回答以下法律问题。

### 问题：
如果一起案件是由人民检察院审查移送起诉的，需要查明哪些问题？

### 响应：
<think>
根据刑事诉讼法的规定，人民检察院在审查移送起诉时，需要对案件进行全面审查，包括犯罪事实、证据、程序、法律适用等方面。具体包括审查犯罪嫌疑人是否符合起诉条件，是否犯罪事实清楚，证据确实、充分，是否需要对侦查活动进行监督，以及是否需要退回补充侦查等。
</think>
人民检察院在审查移送起诉时，应查明以下问题：(一)犯罪嫌疑人身份状况是否清楚，是否符合起诉条件；(二)犯罪事实、情节是否清楚，是否符合起诉条件；(三)侦查活动是否合法，是否需要纠正；(四)是否需要对侦查活动进行监督；(五)犯罪嫌疑人是否认罪认罚，是否需要签署认罪认罚具结书；(六)是否需要退回补充侦查；(七)其他需要查明的事项。<｜end▁of▁sentence｜>

如果一起案件是由人民检察院审查移送起诉的，需要查明哪些问题？



<think>
根据刑事诉讼法的规定，人民检察院在审查移送起诉时，需要对案件进行全面审查，包括犯罪事实、证据、程序、法律适用等方面。具体包括审查犯罪嫌疑人是否符合起诉条件，是否犯罪事实清楚，证据确实、充分，是否需要对侦查活动进行监督，以及是否需要退回补充侦查等。
</think>
人民检察院在审查移送起诉时，应查明以下问题：(一)犯罪嫌疑人身份状况是否清楚，是否符合起诉条件；(二)犯罪事实、情节是否清楚，是否符合起诉条件；(三)侦查活动是否合法，是否需要纠正；(四)是否需要对侦查活动进行监督；(五)犯罪嫌疑人是否认罪认罚，是否需要签署认罪认罚具结书；(六)是否需要退回补充侦查；(七)其他需要查明的事项。<｜end▁of▁sentence｜>


In [12]:
new_sft_model = "DeepSeek-R1-Law-CoT-14B"
model.save_pretrained(new_sft_model) # Local saving
tokenizer.save_pretrained(new_sft_model)
model.save_pretrained_merged(new_sft_model, tokenizer, save_method = "merged_16bit")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 68.37 out of 125.49 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 25%|██▌       | 12/48 [00:00<00:00, 37.24it/s]
We will save to Disk and not RAM now.
100%|██████████| 48/48 [01:02<00:00,  1.30s/it]


Unsloth: Saving tokenizer... Done.
Done.
